In [14]:
import ipywidgets as ipyw
import traitlets as trt
from ipyradiant import LoadWidget


In [13]:
import logging
import re
from pandas import DataFrame
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery
# pattern used to identify bindings in a sparql string
BINDING_PATTERN = re.compile(r"\?([\w]*)")
class SPARQLQueryFramer:
    """A generic Class for building and running SPARQL queries with rdflib.
    TODO need test
    :param initNs: a dict of namespace {term: namespace} to use in rdflib prepareQuery
    :param classBindings: a dict of bindings to set at the class level
        (independent of initBindings).
    :param sparql: a SPARQL parse-able string to use during query
    :param index: an index list to use when building a query result DataFrame
    :param columns: a list of strings to use as column headers for
        the query result DataFrame
    :param query: a valid rdflib Query object
    """
    initNs = {}
    classBindings = {}
    sparql = ""
    index = []
    columns = None
    query = None
    # low cost trait (previous cls.sparql state)
    p_sparql = ""
    @classmethod
    def print_vars(cls) -> None:
        """Utility function to print variables that may be used as bindings
        """
        logging.info("Only variables in the SELECT line are printed.")
        tmp_graph = Graph()
        # Run fake query to print vars
        if not cls.query:
            tmp_query = prepareQuery(cls.sparql, initNs=cls.initNs)
            tmp_res = tmp_graph.query(tmp_query)
        else:
            tmp_res = tmp_graph.query(cls.query)
        print("Vars:\n", sorted([str(var) for var in tmp_res.vars]))
    @classmethod
    def print_potential_bindings(cls) -> None:
        """Utility function to print bindings in the sparql string.
        Note, this method is regex-based, and may not be 100% accurate.
        """
        if not cls.sparql:
            print("No sparql string set in class.")
            return None
        logging.warning("Bindings are not guaranteed to be 100% accurate")
        potential_bindings = [
            str(binding) for binding in set(BINDING_PATTERN.findall(cls.sparql))
        ]
        print("Potential bindings:\n", sorted(potential_bindings))
        return None
    @classmethod
    def run_query(
        cls, graph: Graph, initBindings: dict = None, **initBindingsKwarg,
    ) -> DataFrame:
        """
            Runs a query with optional initBindings, and returns the results
            as a pandas.DataFrame.
        :param graph: the rdflib.graph.Graph to be queried
        :param initBindings: a dictionary of bindings where the key is the variable in
            the sparql string, and the value is the URI/Literal to BIND to the variable.
        :param initBindingsKwarg: kwarg version of initBindings
        :return: pandas.DataFrame containing the contents of the SPARQL query
            result from rdflib
        """
        assert (
            cls.query or cls.sparql
        ), "No rdflib Query or SPARQL string has been set for the class."
        # Check if query should be updated due to stale sparql string
        update_query = cls.p_sparql != cls.sparql
        if not cls.query or update_query:
            cls.query = prepareQuery(cls.sparql, initNs=cls.initNs)
        # note: merge method kwargs with default class bindings
        if initBindings:
            all_bindings = {**cls.classBindings, **initBindings, **initBindingsKwarg}
        else:
            all_bindings = {**cls.classBindings, **initBindingsKwarg}
        result = graph.query(cls.query, initBindings=all_bindings)
        if cls.columns is None:
            # Try to infer from query vars
            try:
                cls.columns = [str(var) for var in result.vars]
            except TypeError:
                # no columns. Probably an ASK or CONSTRUCT query
                logging.debug(
                    "No columns passed, and unable to infer. "
                    "Therefore, no columns were assigned to the DataFrame."
                )
        df = DataFrame(result, columns=cls.columns)
        # update low cost trait
        cls.p_sparql = cls.sparql
        if cls.index:
            return df.set_index(cls.index)
        return df

In [24]:
class PredicateFilterQuery(SPARQLQueryFramer):
    sparql='''
    SELECT DISTINCT ?p ?check
    WHERE {
        ?s ?p ?o .
        BIND(isLiteral(?o) AS ?check)
    }
    '''
    columns = ['predicate','isLiteral']

In [7]:
lw = LoadWidget()
lw

In [25]:
predicate_metadata = PredicateFilterQuery.run_query(lw.graph)

In [2]:
#create sample data
sample_data = [
    'Auba','Laca','Pepe','Dani','Hector','Saka','Granit','Torreira','Saliba','Willian','Coutinho','Mikel','Nelson',
    'Sokratis','Emi','Bernd','Cedric','Tierney','Sead','David','Rob','Joe','Calum','Mesut'
]

In [3]:
#now do with traitlets... yay

class PredicateSelectionWidget(ipyw.HBox):
    available_predicates = trt.Instance(ipyw.SelectMultiple)
    predicates_to_collapse = trt.Instance(ipyw.SelectMultiple)
    add_button = trt.Instance(ipyw.Button)
    remove_button= trt.Instance(ipyw.Button)
    collapse_literals = trt.Instance(ipyw.Checkbox)
    show_literals = trt.Instance(ipyw.Checkbox)
    data = trt.List()
    available_preds_list = trt.List()
    remove_preds_list = trt.List(default_value=[])
    output = ipyw.Output()
    
    @trt.default('available_preds_list')
    def _make_available_preds_list(self):
        return self.data
    
    @trt.default('available_predicates')
    def _make_available_predicates(self):
        pred_selector = ipyw.SelectMultiple(
            options = self.available_preds_list,
            disabled = False,
            layout = ipyw.Layout(height = '300px')
        )
        return pred_selector

    @trt.default('predicates_to_collapse')
    def _make_preds_to_collapse(self):
        preds_to_collapse = ipyw.SelectMultiple(
            options = self.remove_preds_list,
            disabled=False,
            layout = ipyw.Layout(height = '300px')
        )
        return preds_to_collapse

    @trt.default('add_button')
    def _make_add_button(self):
        return ipyw.Button(
            description='Add',
        )
    
    @trt.default('remove_button')
    def _make_remove_button(self):
        return ipyw.Button(
            description='Remove'
        )
    
    @trt.default("collapse_literals")
    def _make_collapse_lits(self):
        return ipyw.Checkbox(
            description = 'Collapse Literals?'
        )
    
    @trt.default("show_literals")
    def _make_show_lits(self):
        return ipyw.Checkbox(
            description = 'Show Literals?'
        )
    
    
    def on_add_clicked(self,*args):
        items_to_move = self.available_predicates.value
        for item in items_to_move:
            self.available_preds_list.remove(item)
            self.remove_preds_list.append(item)
        self.available_predicates.options = self.available_preds_list
        self.predicates_to_collapse.options = self.remove_preds_list
        
    def on_remove_clicked(self,*args):
        items_to_move = self.predicates_to_collapse.value
        for item in items_to_move:
            self.remove_preds_list.remove(item)
            self.available_preds_list.append(item)
        self.available_predicates.options = self.available_preds_list
        self.predicates_to_collapse.options = self.remove_preds_list

    
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.data = kwargs['data']
        
        self.column_one=ipyw.VBox(children=[ipyw.HTML('<h1>Available Predicates</h1>'),self.available_predicates,self.collapse_literals])
        self.column_two=ipyw.VBox(children=[self.add_button,self.remove_button])
        self.column_three=ipyw.VBox(children=[ipyw.HTML('<h1>Predicates to Collapse</h1>'),self.predicates_to_collapse,self.show_literals])
        
        self.add_button.style.button_color = 'lightgreen'
        self.remove_button.style.button_color = 'red'
        self.add_button.layout = ipyw.Layout(margin='100px 0px 0px 0px')
        self.remove_button.layout =ipyw.Layout(margin='0px 0px 0px 0px')
        
        self.add_button.on_click(self.on_add_clicked)
        self.remove_button.on_click(self.on_remove_clicked)
        self.children=[self.column_one,self.column_two,self.column_three]

        

In [27]:
sw = PredicateSelectionWidget(data=list(predicate_metadata.predicate))

In [28]:
sw

PredicateSelectionWidget(children=(VBox(children=(HTML(value='<h1>Available Predicates</h1>'), SelectMultiple(…

In [73]:
sw.output

Output()